# PlusMinus QNN — Blind Quantum Computing

Este notebook demonstra como treinar um modelo de Rede Neural Quântica (QNN)
usando o Qiskit Machine Learning e circuitos gerados a partir do dataset PlusMinus.

In [ ]:
#Instale as dependências 
!pip install pennylane-datasets
!pip install qiskit qiskit-aer qiskit-machine-learning pennylane scikit-learn numpy torch matplotlib

ERROR: Could not find a version that satisfies the requirement install (from versions: none)
ERROR: No matching distribution found for install


In [ ]:

import os
import random
import numpy as np
import torch

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import pennylane as qml

from qiskit import QuantumCircuit
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import Estimator
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.connectors import TorchConnector

import torch.nn as nn
import torch.optim as optim


# Controle de Aleatoriedade e Reprodutibilidade

In [ ]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Rede

In [ ]:
rede = None
def safe_setup_quantumnet():
    global rede
    try:
        from quantumnet.components import Network, Logger
        rede = Network()
        rede.set_ready_topology('grade', 8, 3, 3)
        Logger.activate(Logger)
        print("[quantumnet] Rede inicializada.")
    except Exception as e:
        print(f"[quantumnet] Não disponível ou falhou ao iniciar: {e}")
        rede = None

def enviar_circuito_por_epoca(circuito, epoch, num_qubits, circuit_depth):
    if rede is None:
        return  # silencioso se não houver rede
    try:
        print(f"[Epoch {epoch+1}] Enviando circuito para a rede...")
        rede.application_layer.run_app(
            "BFK_BQC",
            alice_id=6,
            bob_id=0,
            num_qubits=num_qubits,
            scenario=2,
            circuit_depth=circuit_depth,
            circuit=circuito
        )
        rede.start_eprs(5)
        print(f"[Epoch {epoch+1}] Envio concluído.")
    except Exception as e:
        print(f"[Epoch {epoch+1}] Erro ao enviar circuito: {str(e)}")

safe_setup_quantumnet()

# Carregar e preparar o dataset Plus-Minus

In [ ]:
[ds] = qml.data.load("plus-minus")

X = ds.img_train
y = ds.labels_train
X_test = ds.img_test
y_test = ds.labels_test

# Flatten imagens: (n_amostras, altura*largura)
X = X.reshape((X.shape[0], -1))
X_test = X_test.reshape((X_test.shape[0], -1))

# PCA -> reduzir para 4 dimensões (4 qubits)
pca = PCA(n_components=4, random_state=SEED)
X = pca.fit_transform(X)
X_test = pca.transform(X_test)

# Normalização para [0,1]
scaler = MinMaxScaler((0, 1))
X = scaler.fit_transform(X)
X_test = scaler.transform(X_test)

# Split train/val estratificado
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=SEED, stratify=y
)

# Converter rótulos de {-1, +1} para {0, 1}
def to_01(arr):
    arr = np.array(arr).astype(np.float32)
    return ((arr > 0).astype(np.float32)).reshape(-1, 1)

y_train = to_01(y_train)
y_val = to_01(y_val)
y_test = to_01(y_test)

# Tensores Torch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)  # shape (N,1)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Configurar QNN (EstimatorQNN + TorchConnector)

In [ ]:

num_qubits = X.shape[1]  # deve ser 4 após o PCA

feature_map = ZZFeatureMap(num_qubits)
ansatz = RealAmplitudes(num_qubits, reps=1)

# Observável Z...Z (medição no eixo Z em todos os qubits)
observable = SparsePauliOp("Z" * num_qubits)

# Simulador com ruído
noise_model = NoiseModel()
error_1q = depolarizing_error(0.01, 1)
error_2q = depolarizing_error(0.02, 2)
# Conjunto mínimo de portas afetas por ruído (ajuste conforme necessário)
noise_model.add_all_qubit_quantum_error(error_1q, ['h','x','y','rx','ry','rz'])
noise_model.add_all_qubit_quantum_error(error_2q, ['cx'])

simulator = AerSimulator(noise_model=noise_model, shots=1024)

# Estimator usando o backend Aer (para refletir o ruído)
# OBS.: versões recentes permitem passar via options={"backend": simulator}
estimator = Estimator(options={"backend": simulator})

# Circuito base (feature map + ansatz)
qc_base = QuantumCircuit(num_qubits)
qc_base.compose(feature_map, inplace=True)
qc_base.compose(ansatz, inplace=True)

qnn = EstimatorQNN(
    circuit=qc_base,
    input_params=feature_map.parameters,
    weight_params=ansatz.parameters,
    observables=observable,
    estimator=estimator,
    input_gradients=True
)

# Conecta QNN ao PyTorch
model = TorchConnector(qnn)

# Treinamento

In [ ]:

loss_func = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

EPOCHS = 20
print("\nTREINANDO QNN...")

for epoch in range(EPOCHS):
    model.train()
    optimizer.zero_grad()

    # Forward
    output = model(X_train_tensor)             # shape (N, 1)
    probs = torch.sigmoid(output)              # garantir [0,1]
    loss = loss_func(probs.view(-1, 1), y_train_tensor.view(-1, 1))

    # Backprop
    loss.backward()
    optimizer.step()

    # Validação periódica
    if (epoch + 1) % 5 == 0:
        model.eval()
        with torch.no_grad():
            val_output = model(X_val_tensor)
            val_probs = torch.sigmoid(val_output).view(-1)
            val_pred = (val_probs >= 0.5).float().cpu().numpy()
            val_true = y_val_tensor.view(-1).cpu().numpy()
            val_acc = accuracy_score(val_true, val_pred)
        print(f"Epoch {epoch+1:02d}: Loss = {loss.item():.4f} | Val_Acc = {val_acc:.4f}")

    # (Opcional) enviar circuito treinado para a rede
    with torch.no_grad():
        trained_weights = model.weight.detach().cpu().numpy()
    final_circuit = QuantumCircuit(num_qubits)
    final_circuit.compose(feature_map, inplace=True)
    final_circuit.compose(ansatz.assign_parameters(trained_weights), inplace=True)
    enviar_circuito_por_epoca(
        circuito=final_circuit,
        epoch=epoch,
        num_qubits=final_circuit.num_qubits,
        circuit_depth=final_circuit.depth()
    )

# Avaliação final

In [ ]:

print("\nAVALIAÇÃO FINAL...")
model.eval()
with torch.no_grad():
    preds = model(X_test_tensor)
    probs = torch.sigmoid(preds).view(-1)
    y_pred = (probs >= 0.5).float().cpu().numpy().astype(int)
    y_true = y_test_tensor.view(-1).cpu().numpy().astype(int)

acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred, zero_division=0)
rec = recall_score(y_true, y_pred, zero_division=0)
f1 = f1_score(y_true, y_pred, zero_division=0)

print("\nMÉTRICAS NO TESTE:")
print(f"Acurácia : {acc:.4f}")
print(f"Precisão : {prec:.4f}")
print(f"Recall   : {rec:.4f}")
print(f"F1 Score : {f1:.4f}")


Hosts inicializados
Canais inicializados
Pares EPRs adicionados
[quantumnet] Rede inicializada.


/tmp/ipykernel_33761/3852483226.py:150: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator(options={"backend": simulator})
/tmp/ipykernel_33761/3852483226.py:157: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = EstimatorQNN(



TREINANDO (EstimatorQNN + TorchConnector)...


2025-08-28 16:36:13,657: Protocolo configurado para 2 rodadas.
2025-08-28 16:36:13,658: Timeslot 1. Iniciando protocolo BFK com 4 qubits, 2 rodadas, e cenário 2.
2025-08-28 16:36:13,658: Memória do cliente 6 (Alice) limpa com sucesso.
2025-08-28 16:36:13,659: Memória do servidor 0 (Bob) limpa com sucesso.
2025-08-28 16:36:13,660: Timeslot 2.
2025-08-28 16:36:13,660: Qubit 104 preparado pelo cliente 6.
2025-08-28 16:36:13,661: Qubit 367 preparado pelo cliente 6.
2025-08-28 16:36:13,662: Qubit 826 preparado pelo cliente 6.
2025-08-28 16:36:13,662: Qubit 549 preparado pelo cliente 6.
2025-08-28 16:36:13,663: Calculando rota padrão para o transporte.
2025-08-28 16:36:13,663: Timeslot 3: Buscando rota válida entre 6 e 0.
2025-08-28 16:36:13,665: Rota válida encontrada: [6, 3, 0]
2025-08-28 16:36:13,666: Limpando pares EPRs residuais na rota: [6, 3, 0]
2025-08-28 16:36:13,666: Pares EPRs limpos no segmento 6 -> 3.
2025-08-28 16:36:13,667: Pares EPRs limpos no segmento 3 -> 0.
2025-08-28 16:3

[Epoch 1] Enviando circuito para a rede...
Tempo de Operação: 2
Pares EPRs adicionados
[Epoch 1] Envio concluído.


...

[Epoch 18] Enviando circuito para a rede...
Tempo de Operação: 2
Pares EPRs adicionados
[Epoch 18] Envio concluído.


2025-08-28 17:14:44,539: Protocolo configurado para 2 rodadas.
2025-08-28 17:14:44,540: Timeslot 289. Iniciando protocolo BFK com 4 qubits, 2 rodadas, e cenário 2.
2025-08-28 17:14:44,541: Memória do cliente 6 (Alice) limpa com sucesso.
2025-08-28 17:14:44,542: Memória do servidor 0 (Bob) limpa com sucesso.
2025-08-28 17:14:44,542: Timeslot 290.
2025-08-28 17:14:44,543: Qubit 476 preparado pelo cliente 6.
2025-08-28 17:14:44,543: Qubit 330 preparado pelo cliente 6.
2025-08-28 17:14:44,544: Qubit 285 preparado pelo cliente 6.
2025-08-28 17:14:44,544: Qubit 768 preparado pelo cliente 6.
2025-08-28 17:14:44,545: Calculando rota padrão para o transporte.
2025-08-28 17:14:44,545: Timeslot 291: Buscando rota válida entre 6 e 0.
2025-08-28 17:14:44,547: Rota válida encontrada: [6, 3, 0]
2025-08-28 17:14:44,547: Limpando pares EPRs residuais na rota: [6, 3, 0]
2025-08-28 17:14:44,548: Pares EPRs limpos no segmento 6 -> 3.
2025-08-28 17:14:44,548: Pares EPRs limpos no segmento 3 -> 0.
2025-08-2

[Epoch 19] Enviando circuito para a rede...
Tempo de Operação: 2
Pares EPRs adicionados
[Epoch 19] Envio concluído.


2025-08-28 17:16:54,159: Protocolo configurado para 2 rodadas.
2025-08-28 17:16:54,160: Timeslot 305. Iniciando protocolo BFK com 4 qubits, 2 rodadas, e cenário 2.
2025-08-28 17:16:54,161: Memória do cliente 6 (Alice) limpa com sucesso.
2025-08-28 17:16:54,161: Memória do servidor 0 (Bob) limpa com sucesso.
2025-08-28 17:16:54,162: Timeslot 306.
2025-08-28 17:16:54,163: Qubit 95 preparado pelo cliente 6.
2025-08-28 17:16:54,163: Qubit 708 preparado pelo cliente 6.
2025-08-28 17:16:54,164: Qubit 484 preparado pelo cliente 6.
2025-08-28 17:16:54,164: Qubit 435 preparado pelo cliente 6.
2025-08-28 17:16:54,165: Calculando rota padrão para o transporte.
2025-08-28 17:16:54,165: Timeslot 307: Buscando rota válida entre 6 e 0.
2025-08-28 17:16:54,166: Rota válida encontrada: [6, 3, 0]
2025-08-28 17:16:54,167: Limpando pares EPRs residuais na rota: [6, 3, 0]
2025-08-28 17:16:54,167: Pares EPRs limpos no segmento 6 -> 3.
2025-08-28 17:16:54,168: Pares EPRs limpos no segmento 3 -> 0.
2025-08-28

Epoch 20: Loss = 0.7378 | Val_Acc = 0.3700
[Epoch 20] Enviando circuito para a rede...
Tempo de Operação: 2
Pares EPRs adicionados
[Epoch 20] Envio concluído.

AVALIAÇÃO FINAL (Teste)...

MÉTRICAS NO TESTE:
Acurácia : 0.4650
Precisão : 0.8028
Recall   : 0.3800
F1 Score : 0.5158
